In [71]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))
import talib
import sqlite3 as sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.font_manager as fm
from mplfinance.original_flavor import candlestick_ohlc
from util.util import get_ucodes

In [72]:
path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
print(path_sqlite)
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()
conn.row_factory = lambda cursor, row: row[0]
no_days = 60

D:\PycharmProjects\scrapy-001\data\aastock\sqlite\hk-marketwatch.db


In [73]:
data1 = {}
no_days = 50
ucodes = ['hsi', 'hsce', 'hstech', 'ssec', 'csi300', 'sp500', 'dji', 'ixic']
for ucode in ucodes:
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC LIMIT {}) AS t 
                            ORDER BY t.stime """.format(ucode, no_days)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data1[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data1[ucode].index = pd.to_datetime(data1[ucode].sdate)
    data1[ucode]['weekday'] = data1[ucode].index.dayofweek
    data1[ucode]['pchng'] = data1[ucode]['last'].pct_change()*100
conn.close()

In [74]:
for ucode, df in data1.items():
    for i in range(5):
        print(ucode, i, df.loc[(df.weekday == i)]['pchng'].sum())

hsi 0 0.8823416355704605
hsi 1 1.9281502723071053
hsi 2 -0.5506325167501047
hsi 3 0.710934823131903
hsi 4 -7.392260805866535
hsce 0 -0.47040089305587784
hsce 1 0.41170332598552406
hsce 2 -0.3700704533275867
hsce 3 -0.3776475250366622
hsce 4 -7.1170144084869555
hstech 0 -10.124935179847151
hstech 1 6.041077706101261
hstech 2 -1.1386640766179124
hstech 3 -2.4438923491845177
hstech 4 -5.271143919707965
ssec 0 -0.8395129613748109
ssec 1 -0.7395933413590683
ssec 2 -0.9788609426828465
ssec 3 2.2056325976594127
ssec 4 -1.7341336990994827
csi300 0 -4.574752621935085
csi300 1 -1.0907404750717609
csi300 2 -0.933001752239615
csi300 3 1.2076133583463493
csi300 4 -2.9219719423539403
sp500 0 6.1237012574514855
sp500 1 0.621370472210514
sp500 2 0.7168793930374839
sp500 3 -1.2875112699712647
sp500 4 1.9231725801122312
dji 0 5.284311048450729
dji 1 -2.0707415582515987
dji 2 3.258742107480417
dji 3 -1.8319455223195114
dji 4 2.0248670777788913
ixic 0 3.2629713811976324
ixic 1 5.835979007715597
ixic 2 -4.